# Fine Tune Com Wikipédia



### Descriçao

---
1. O objetivo deste notebook é aprender sobre Fine Tune
2. Vamos descorrer sobre modelos de LLM e utilizar a biblioteca Torch para descorrer sobre assunto
3. Vamos ensinar nossa máquina com os dados do [Wikipédia](https://pt.wikipedia.org/wiki/Ci%C3%AAncia)
---

### Baixando os pacotes




In [ ]:
!pip install transformers transformers[torch] datasets

### Documentação

1. **Transformers**:
   - [Documentação do Hugging Face Transformers](https://huggingface.co/models?sort=trending)
   - [Repositório GitHub](https://github.com/huggingface/transformers)

2. **Requests**:
   - [Documentação do Requests](https://docs.python-requests.org/en/latest/)
   - [Repositório GitHub](https://github.com/psf/requests)

3. **Datasets**:
   - [Documentação do Hugging Face Datasets](https://huggingface.co/docs/datasets/)
   - [Repositório GitHub](https://github.com/huggingface/datasets)

4. **Torch**:
   - [Documentação do PyTorch](https://pytorch.org/docs/stable/index.html)
   - [Repositório GitHub](https://github.com/pytorch/pytorch)


### Importando as bibliotecas

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import requests
from datasets import Dataset, load_dataset #esta biblioteca ajuda a carregar os dados
import torch

### importando o modelo

In [ ]:
model_name = "pierreguillou/gpt2-small-portuguese" #Carregamos o modelo
tokenizer = GPT2Tokenizer.from_pretrained(model_name) # criamos um tokenizador (PLN)
model = GPT2LMHeadModel.from_pretrained(model_name) # Carregamos o modelo GP2, pre treinado

# Requisitando o Wikipedia

In [ ]:
url = "https://pt.wikipedia.org/wiki/The_Hitchhiker%27s_Guide_to_the_Galaxy" # para fins de pesquisa, eh bom sempre pegar apenas uma unica pagina. Ao inves da api completa do Wikipedia
response = requests.get(url)
text_data = response.text
text_data

In [ ]:

lines = text_data.split('\n')
data_dict = {"text": lines}

dataset = Dataset.from_dict(data_dict)
dataset

#### Tokenização

In [ ]:
# tokenização do dataset
def token_map(examples):
    return tokenizer(examples["text"], # extrai o nosso dataset
                     padding="max_length", #preenche sequencias até o comprimento máximo
                     truncation=True, #Trunca, caso exceda o máximo de sequencias
                     max_length=128) #define o cumprimento de cada token em 128

tokenized_dataset = dataset.map(token_map, batched=True)

In [ ]:
tokenized_dataset

In [ ]:
tokenized_dataset[400]['text'] #texto original do wikipedia
#tokenized_dataset[200]['input_ids'] # ids do tokens do texto original.
#okenized_dataset[100]['attention_mask'] # Lista de tokens que o modelo vai prestar atencao.


In [ ]:
# Preparando o lote de dados
#Data Collector é um utilitário usado para preparar os dados durante a fase de treino

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, #aqui eu passo o tokenizador que criamos nas primeiras celulas
                                                mlm=False) #mlm significa Masket Language Model, este parametro só é usado em casos onde o transformers nao usa mascara para treinar

In [ ]:
# hiper parâmetros

training_args = TrainingArguments(
    output_dir="./output", #diretorio de saida
    overwrite_output_dir=True, #Sobreescrever o diretorio
    num_train_epochs=1, #  Este epoch, ou epoca, eh apenas para fins de processamento e para apresentacao da aula.
    per_device_train_batch_size=32, #tamanho do modelo em batchs
    save_steps=10_000, #indica quantas etapas os modelos e arquivos serao salvos
    save_total_limit=2 #limita o numero de checkpoints
)

### criando o treinamento

In [ ]:
trainer = Trainer(model=model, #modelo de linguagem
                  args=training_args, #os hiperparamentros
                  data_collator=data_collator,  # o utilitario
                  train_dataset=tokenized_dataset) #a customizacao

#### Treinando

In [ ]:
trainer.train()

### Salvando o modelo

In [ ]:
model.save_pretrained("meu_guia")
tokenizer.save_pretrained("meu_guia")

# Utilizando o Modelo

#importando a biblioteca que criamos

In [ ]:
from transformers import pipeline

In [ ]:
gerador = pipeline("text-generation", model="meu_guia")
instrucao = """Me fale sobre o Maycon"""

resultado = gerador(instrucao,
                    truncation=True,
                    max_length=600, #maximo de comprimento do texto
                    do_sample=True, #retorna uma amostragem estocagem, mais diversificada quando true
                    temperature=0.9, #vai de zero a 1 e quanto maior, mais criativa fica o texto. tmb define a tal alucinacao
                    top_k=50, # limita as proximas palavras ao numero k
                    top_p=0.85#limita o conjunto de palavras para um novo grupo
                   ) #ID do token de padding do GPT-2

In [ ]:
resultado = resultado[0]['generated_text']
resultado

In [ ]:
indice_fim_texto_original = resultado.find(instrucao)
novo_texto = resultado[indice_fim_texto_original + len(instrucao):].strip().replace('\n\n', ' ')
novo_texto